In [1]:
# Load packages
import numpy as np
import xarray as xr
from pathlib import Path
import cftime
import pandas as pd
from matplotlib import pyplot as plt
import os, re, glob

In [2]:
def plev3d(raw_data):
# Info for function
    """
    Function: Compute monthly or seasonal normals for wrfplev3d
    
    Input: raw_data in xarray dataset 
    
    Output: xarray dataset grouped by
    """
# Build empty dictionary
    raw_dataarray={}
# Compute normals by variables
    for var in list(raw_data.data_vars):
# Skip for times
        if 'Times' in var:
            pass
        else:
            raw_dataarray[var]=raw_data[var].compute()
# Merge dataarrays back into 1 dataset
    first_loop=True
    for key in raw_dataarray.keys():
        if first_loop:
            output=raw_dataarray[key]
            first_loop=False
        else:
            output=xr.merge([output,raw_dataarray[key]])
    return output

In [3]:
def build_parameter(dir_input):
    """
    
    Function: Extract parameter and start year from directory
             based on types
    
    Input: Folder directory as strings
    
    Output: parameter and start year as strings
    """
# Grab based on directory type
# Start year  
    time=open(dir_input+'stepfile')
    lines=time.readlines()
    start_year=lines[0][0:4]
    if 'WRFTools' in dir_input:
# Parameter
        par_start=dir_input.find('WRFTools')
        par=dir_input[par_start:-1]
    else:
# parameter
        par_start=dir_input.find('wrf/')+4
        par_end=dir_input.find(start_year)-1
        par=dir_input[par_start:par_end]
    return par,start_year

In [4]:
def climate_normals(dir_input,sub_dir='wrfavg',dir_ouput=None,freq='M'):
# Info for function 
    """
    Function:Compute monthly or seasonal normals and create netcdf 
             from monthly average data
                
    Input arguments: directories as strings
                    Frequency: M for monthly S for Seasonal
    
    Output Directory:<Current working directory>/<pararmeter>/
                     monthly_norm_<start year>/
                     by default unless specified otherwise
    
    Input data: monthly average netcdf with dask
    
    Output data: monthly normals netcdf with dask
    """
# Extract input info
    par,start_year=build_parameter(dir_input)

# Open datasets with dask
    raw_data={file.stem :xr.open_dataset(file,chunks={'time':-1},decode_times=False) for file in Path(dir_input).glob(sub_dir+'/*monthly.nc')}
    
# Compute Normals
    First_loop=True
    for key in raw_data.keys():
        print('\n',key,'start')
        data=raw_data[key]
        
# Convert time to datetime64
        data['time'] = pd.date_range(start=start_year+'-01-01', periods=data.sizes['time'], freq='MS')
# Find End Year    
        if First_loop:
            end_year=str(data.time[-1].values)[0:4]
# Create output directory
            if dir_ouput == None:
# Use current working directory by default
                cwd_start=os.getcwd().find('/scratch')
                cwd=os.getcwd()
            if freq == 'M':
                out_dir=os.path.join(cwd[cwd_start::],par+'/monthly_norm_'+start_year+'_'+end_year)
            elif freq == 'S':
                out_dir=os.path.join(cwd[cwd_start::],par+'/seasonal_norm_'+start_year+'_'+end_year)
            else:
                if freq == 'M':
                    out_dir=os.path.join(dir_ouput,par+'/monthly_norm_'+start_year+'_'+end_year)
                elif freq == 'S':
                    out_dir=os.path.join(dir_ouput,par+'/seasonal_norm_'+start_year+'_'+end_year)
# Check if directory exists
            if os.path.exists(out_dir)==False:
                os.makedirs(out_dir)
            else:
                print('Directory exists\n')
            First_loop=False
# # Check if file exists
        if freq =='M' and os.path.isfile(out_dir+'/'+key+'_mon_norm.nc'):
            print(key,'File exists\n')
        elif freq =='S' and os.path.isfile(out_dir+'/'+key+'_sea_norm.nc'):
            print(key,'File exists\n')
        elif 'tmp_wrfavg_proc01' in key:
            print(key,'skip for now\n')
        else:
            if freq =='M':
                data_norm=data.groupby('time.month').mean('time')
            elif freq == 'S':
                data_norm=data.groupby('time.season').mean('time')
# Compute by dataarray for plev.nc
            if 'plev' in key:
                data_norm=plev3d(data_norm)
            else:
                pass
# Create netcdf for monthly normals
            if freq=='M':
                data_norm.to_netcdf(out_dir+'/'+key+'_mon_norm.nc')
            elif freq =='S':
                data_norm.to_netcdf(out_dir+'/'+key+'_sea_norm.nc')
            print('done\n')


In [5]:
# Run all dirs
dirs=[
    '/home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_S37_na24_conus43-flk_2039/',
    '/home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_S37_na24_conus43-flk_2079/',
    '/home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_HS_na24_new43loc-flk_1979/',
    '/home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_HS_na24_new43-flk_1979/',
    '/scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43-flk_2039/',
    '/scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43-flk_2079/',
    '/scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43loc-flk_2039/',
    '/scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43loc-flk_2079/',
    '/scratch/a/aerler/aerler/Mani/wrf/ERA5_HS_NA24_new43-flk-1979/',
    '/scratch/a/aerler/aerler/Mani/wrf/ERA5_HS_NA24_new43loc-flk-1979/',
    '/scratch/a/aerler/aerler/Mani/ERAI_AND_ERA5_RUN_CASES/WRFTools_RC87/',
    '/scratch/a/aerler/aerler/Mani/ERAI_AND_ERA5_RUN_CASES/WRFTools_RC88/',
    '/scratch/a/aerler/aerler/Mani/ERAI_AND_ERA5_RUN_CASES/WRFTools_RC91/',
    '/scratch/a/aerler/aerler/Mani/ERAI_AND_ERA5_RUN_CASES/WRFTools_RC92/'
]

In [6]:
for dir_a in dirs:
    print(dir_a,'\nstart\n')
    a=climate_normals(dir_a,sub_dir='wrfavg',freq='S')
    print('\n',dir_a,'\ndone\n')

/home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_S37_na24_conus43-flk_2039/ 
start


 wrfhydro_d01_monthly start
Directory exists

wrfhydro_d01_monthly File exists


 wrfplev3d_d01_monthly start
wrfplev3d_d01_monthly File exists


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 wrfrad_d01_monthly start
wrfrad_d01_monthly File exists


 /home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_S37_na24_conus43-flk_2039/ 
done

/home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_S37_na24_conus43-flk_2079/ 
start


 wrfhydro_d01_monthly start
Directory exists

wrfhydro_d01_monthly File exists


 wrfplev3d_d01_monthly start
wrfplev3d_d01_monthly File exists


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 wrfrad_d01_monthly start
wrfrad_d01

/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter20

done


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 wrfrad_d01_monthly start
wrfrad_d01_monthly File exists


 /home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_HS_na24_new43loc-flk_1979/ 
done

/home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_HS_na24_new43-flk_1979/ 
start


 wrfhydro_d01_monthly start
Directory exists

wrfhydro_d01_monthly File exists


 wrfplev3d_d01_monthly start
wrfplev3d_d01_monthly File exists


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 wrfrad_d01_monthly start
wrfrad_d01_monthly File exists


 /home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_HS_na24_new43-flk_1979/ 
done

/scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43-flk_2039/ 
start


 wrfhydro_d01_monthly start
Directory exists

/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter20

done


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 wrfrad_d01_monthly start
wrfrad_d01_monthly File exists


 /scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43-flk_2039/ 
done

/scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43-flk_2079/ 
start


 wrfhydro_d01_monthly start
Directory exists

wrfhydro_d01_monthly File exists


 wrfplev3d_d01_monthly start
wrfplev3d_d01_monthly File exists


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 wrfrad_d01_monthly start
wrfrad_d01_monthly File exists


 /scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43-flk_2079/ 
done

/scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43loc-flk_2039/ 
start


 wrfhydro_d01_monthly start
Directory exists

wrfh

/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter20

done


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 wrfrad_d01_monthly start
wrfrad_d01_monthly File exists


 /scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43loc-flk_2079/ 
done

/scratch/a/aerler/aerler/Mani/wrf/ERA5_HS_NA24_new43-flk-1979/ 
start


 wrfhydro_d01_monthly start
Directory exists

wrfhydro_d01_monthly File exists


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 tmp_wrfavg_proc01_wrfplev3d_d01_monthly start
tmp_wrfavg_proc01_wrfplev3d_d01_monthly skip for now


 wrfrad_d01_monthly start
wrfrad_d01_monthly File exists


 /scratch/a/aerler/aerler/Mani/wrf/ERA5_HS_NA24_new43-flk-1979/ 
done

/scratch/a/aerler/aerler/Mani/wrf/ERA5_HS_NA24_new43loc-flk-1979/ 
start


 wrfhydro_d01_monthly start
Dir

In [7]:
for dir_a in dirs:
    print(dir_a,'\nstart\n')
    climate_normals(dir_a,freq='M')
    print('\n',dir_a,'\ndone\n')

/home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_S37_na24_conus43-flk_2039/ 
start


 wrfhydro_d01_monthly start
Directory exists

wrfhydro_d01_monthly File exists


 wrfplev3d_d01_monthly start
wrfplev3d_d01_monthly File exists


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 wrfrad_d01_monthly start
wrfrad_d01_monthly File exists


 /home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_S37_na24_conus43-flk_2039/ 
done

/home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_S37_na24_conus43-flk_2079/ 
start


 wrfhydro_d01_monthly start
Directory exists

wrfhydro_d01_monthly File exists


 wrfplev3d_d01_monthly start
wrfplev3d_d01_monthly File exists


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 wrfrad_d01_monthly start
wrfrad_d01

/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter20

done


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 wrfrad_d01_monthly start
wrfrad_d01_monthly File exists


 /home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_HS_na24_new43loc-flk_1979/ 
done

/home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_HS_na24_new43-flk_1979/ 
start


 wrfhydro_d01_monthly start
Directory exists

wrfhydro_d01_monthly File exists


 wrfplev3d_d01_monthly start
wrfplev3d_d01_monthly File exists


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 wrfrad_d01_monthly start
wrfrad_d01_monthly File exists


 /home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_HS_na24_new43-flk_1979/ 
done

/scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43-flk_2039/ 
start


 wrfhydro_d01_monthly start
Directory exists

/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter20

done


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 wrfrad_d01_monthly start
wrfrad_d01_monthly File exists


 /scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43-flk_2039/ 
done

/scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43-flk_2079/ 
start


 wrfhydro_d01_monthly start
Directory exists

wrfhydro_d01_monthly File exists


 wrfplev3d_d01_monthly start
wrfplev3d_d01_monthly File exists


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 wrfrad_d01_monthly start
wrfrad_d01_monthly File exists


 /scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43-flk_2079/ 
done

/scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43loc-flk_2039/ 
start


 wrfhydro_d01_monthly start
Directory exists

wrfh

/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/local/jupyter20

done


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 wrfrad_d01_monthly start
wrfrad_d01_monthly File exists


 /scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43loc-flk_2079/ 
done

/scratch/a/aerler/aerler/Mani/wrf/ERA5_HS_NA24_new43-flk-1979/ 
start


 wrfhydro_d01_monthly start
Directory exists

wrfhydro_d01_monthly File exists


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 tmp_wrfavg_proc01_wrfplev3d_d01_monthly start
tmp_wrfavg_proc01_wrfplev3d_d01_monthly skip for now


 wrfrad_d01_monthly start
wrfrad_d01_monthly File exists


 /scratch/a/aerler/aerler/Mani/wrf/ERA5_HS_NA24_new43-flk-1979/ 
done

/scratch/a/aerler/aerler/Mani/wrf/ERA5_HS_NA24_new43loc-flk-1979/ 
start


 wrfhydro_d01_monthly start
Dir

In [15]:
lake_dir='/home/p/peltier/mahdinia/scratch/ERAI_AND_ERA5_RUN_CASES'
for folder in glob.glob(lake_dir+'/WRFTools*/'):
    print(folder,'start\n')
    climate_normals(folder,'wrfavg*',freq='M')
    climate_normals(folder,'wrfavg*',freq='S')
    print('\n',folder,'\nend\n')

/home/p/peltier/mahdinia/scratch/ERAI_AND_ERA5_RUN_CASES/WRFTools_RC68b/ start


 wrfhydro_d01_monthly start
Directory exists

wrfhydro_d01_monthly File exists


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 wrfrad_d01_monthly start
wrfrad_d01_monthly File exists


 wrfsnow_d01_monthly start
wrfsnow_d01_monthly File exists


 wrfplev3d_d01_monthly start
wrfplev3d_d01_monthly File exists


 wrfhydro_d01_monthly start
Directory exists

wrfhydro_d01_monthly File exists


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 wrfrad_d01_monthly start
wrfrad_d01_monthly File exists


 wrfsnow_d01_monthly start
wrfsnow_d01_monthly File exists


 wrfplev3d_d01_monthly start
wrfplev3d_d01_monthly File exists


 /home/p/peltier/mahdinia

In [16]:
dir_in='/scratch/p/peltier/mahdinia/wrf/MPIESM12HR_HS_na24_conus43-flk_1979/'
climate_normals(dir_in,sub_dir='wrfavg_1979-2010',freq='M')
climate_normals(dir_in,sub_dir='wrfavg_1979-2010',freq='S')


 wrfhydro_d01_monthly start
Directory exists

wrfhydro_d01_monthly File exists


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 wrfrad_d01_monthly start
wrfrad_d01_monthly File exists


 wrfhydro_d01_monthly start
Directory exists

wrfhydro_d01_monthly File exists


 wrflsm_d01_monthly start
wrflsm_d01_monthly File exists


 wrfsrfc_d01_monthly start
wrfsrfc_d01_monthly File exists


 wrfxtrm_d01_monthly start
wrfxtrm_d01_monthly File exists


 wrfrad_d01_monthly start
wrfrad_d01_monthly File exists

